# Picúr, az éhes hörcsög

Úgy érzitek, hogy most már elég idősek, érettek és felelősség tudatosak vagytok és eljött az ideje, hogy beszerezzetek egy kicsi, de annál cukibb szíriai hörcsögöt. Picúrnak fogjátok hívni. Ő egy nagyon aranyos és virgonc kis hörcsög aki nagyon szereti a répát, de a répa megszerzésében nektek kell segíteni. Nektek kell megmutatni az utat és irányítani Picúrt, hogy megtalálja a répát. 

## Mit fogsz készíteni?

Egy egyszerű számítógépes játékot fogunk készíteni, amiben egy Picúr nevű hörcsögöt fogunk három gomb segítségével irányítani jobbra, balra, le- és felfele. A célja a játéknak, hogy Picúr megtalálja a répát. 

## Mit tanulsz meg?

A játék elkészítésével a következőket tanulod meg:

* Hogyan tudsz több gombot egy áramkörbe beiktatni
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal
* Hogyan kell egy grafikus felületet létrehozni a **pygame** modullal
* Hogyan hangold össze a grafikus megjelenítő felületet a gombokkal és mozgass elemeket a grafikus felületen

## A projekt részletekre bontása

* A **pygame** installálása
* Defininálni a le, fel, jobbra és balra irányokat
* Beszerezni képeket a hörcsögről és a répáról
* Elhelyezni ezeket a játéktéren
* Megakadályozni, hogy a hörcsög lemehessen a képernyőről
* Frissíteni a képek helyzetét a mozgatások után
* Kiírni a képernyőre, hogy Yummy ha a hörcsög megtalálta a répát

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) 3 db nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

## A kapcsolási rajz

<img src="prog09_circuit.png" width=600 height=400 />

1) A nyomógombot lábait illesszük a breadbord mélyedéseibe. Ügyeljünk arra, hogy a nyomógomb egy (vízszintes) sorba kerülő lábai a gomb lenyomása nélkül is kapcsolatban legyenek egymással, míg a (függőlegesen) egy sorba kerülő lábai csak a lenyomás következtében lesznek összekötve. Mindezt egy műszerrel leellenprízhetjük. 

2) Ugyanezt ismételjük meg még kétszer, ahogy az ábra mutatja. A két szélső gombok szolgálnak majd a következő és az előző zene elindítására, míg a középső kettős funkciót lát majd el, elindítja és leállítja a zenét. 

3) Fogjunk 3 jumper kábelt (pl. kék, zöld és narancssárga) és kössük egyenként egyik végüket a gomb egyik lábának sorába. A kék kábel másik végét a Raspberry Pi **GPIO 02**-es (vagy **3**-as) jelöléssel ellátott tüskéjére, a zöld kábel másik végét a **GPIO 03**-es (vagy **5**-ös) tüskére és a narancssárga kábel másik végét a **GPIO 17**-es (vagy **11**-es) tüskére kössük. 

4) A gombok másik lábának sorába pedig földelést kell kötni. Fogjunk sárga jumper kábelt és a gombok szabadon lévő lábának sorába tegyük a kábel egyik végét, míg a másikat a szélső függőlegesen összekötött oszlopok egyikébe. Ugyanabba az oszlopba kössünk be egy jumper kábelt, aminek a másik vége a Raspberry Pi **GND** (pl. **34**-es) jelöléssel ellátott tüskéjéhez kapcsolódik. Ez a tüske alapértelmezetten földelés.

## A kód

Ha valaki játékokat szeretne fejleszteni Pythonbó, annak a [*pygame*](https://www.pygame.org/news) modul egy kézenfekvő csomag. Ez lehetővé teszi a felhasználónak, hogy könnyedén és rövid idő alatt multimédiás termékeket hozzon létre. A *pygame* általában alapjáraton fel van installálva a Raspbian operációs rendszeren de jól utána kell járni, hogy milyen Python verzióhoz tartozik.

Amennyiben a te számítógépeden nem lenne felinstallálva a *pygame* (amit a ```pip list``` paranccsal tudsz ellenőrizni), nyiss meg egy terminált és írd be hogy ```pip install pygame```. Ha nem kaptál hibaüzenetet, akkor ezután már rendelkezned kellene evvel a csomaggal. 

A kódban szükségünk lesz még egy hörcsög és egy répa képére is. Ezeket bárhonnan be lehet tölteni az internetrő, de mi a [http://www.freepik.com/](http://www.freepik.com/) weboldalról keresgéltünk.

A kódunk első részében, mint mindig, először beimportálunk néhány modult. A ```gpiozero``` modulból a ```Button``` objektumot ami lehetővé teszi, hogy a Raspi kommunikáljon a gombbal a **GPIO** tüskéken keresztül. Emellett betöltjük a **time** modult amit időmérésre fogunk majd használni, a **random** modult amit a véletlenszerű számok generálására használunk, a **sys** modult, ami lehetővé teszi, hogy pythonból elérjük az operációs rendszer parancsait, a **pygame** csomagot ami a grafikai megjelenítésért felel.

In [ ]:
from gpiozero import Button
import time, random
import sys, pygame

A modulok beolvasása után inicializálunk három ```Button``` objektumot, amiket ```left```-nek, ```right```-nak és ```up_down```-nak nevezünk el. Ezeket a *17*-es, *2*-es és a *3*-ss számú **GPIO** tüskékre csatlakoztatjuk.

In [ ]:
right = Button(2)
up_down = Button(3)
left = Button(17)

A következő lépésben elkezdjük definiálni a grafikus felületünket. A ```pygame.init()``` közli a pythonnal, hogy készek vagyunk használni a grafikus felületet és elkezdjük azt inicializálni. A ```pygame.font.init()``` ugyanezt csinálja, csak a betűtípusokat inicializálja. Ezután létrehozunk egy ```font``` objektumot az operációs rendszerünk betűtípus készletéből a ```pygame.font.SysFont('Comic Sans MS', 30)``` paranccsal. 

In [ ]:
pygame.init( )
pygame.font.init()
font = pygame.font.SysFont('Comic Sans MS', 30)

Ezután definiálunk néhány előre meghatározott értéket a grafikus felületünkkel (canvas) kapcsolatban, mint pl. a szélesség (width), magasság (height), a háttér színe RGB stílusban (white) és a hörcsög lépéseinek nagysága (step).

In [1]:
#Set canvas parameters
width, height = 500, 500
size = width
white = 255,255,255
step = 50

A következő lépésben definiáljuk a grafikus felületet. Először adunk egy címet a grafikus ablaknak, ```pygame.display.set_caption("Running Picur Hamster")``` majd beállítjuk az ablak méretét, ```pygame.display.set_mode(size)``` és a ```screen``` változóhoz rendeljük. 

In [1]:
pygame.display.set_caption("Running Picur Hamster")
screen = pygame.display.set_mode(size)

Ezután betöltjük a képeket, amiket a grafikus ablakban szeretnénk megjeleníteni. Először a hörcsög képével kezdjük. A ```pygame.image.load()``` parancs szolgál a beolvasásra, aminek az argumentumába a kép nevét kell megadni (adott esetben annak lokáviójával együtt). A függvény által visszaadott objektumot a ```ball``` változóban tároljuk. Mivel az általunk letöltött kép túl nagy (ezt mindenki tesztelje le a saját képével), mi lecsökkentjük a méretét úgy, hogy az egy lépés méretével legyen egyenlő, ```pygame.transform.scale(ball, (step,step))```. Az így létrehozott képet (objektumot) szeretnénk majd mozgatni a grafikus felületen. Mivel eddig az objektumnak nem volt pozíciója a felületen, a ```get_rect()``` metódussal inicializáljuk azt (alapértelmezetten, az (x,y) = (0,0) koordinátákra helyezi az objektumot) és a visszaadott pozíció paramétereket a ```ballrect``` változóba mentjük. Majd az így kapott objektum középpontjának *x* és *y* koordinátáit változtatjuk, hogy a hörcsög a grafikus ablak közepén jelenjen meg a ```centerx``` és ```centery``` metódusokkal. 

In [2]:
ball = pygame.image.load("hamster.png")
ball = pygame.transform.scale(ball, (step,step))

ballrect = ball.get_rect()
ballrect.centerx = width/2
ballrect.centery = height/2

Ugyanezeket a lépéseket megcsináljuk a répa képére is némi módosítással. Hogy érdekesebbé tegyük a játékot, minden egyes újrakezdésnél, a répa pozícióját random módon helyezzük el a grafikus képernyőn. Azaz, generálunk egy egész számot, mind az *x* és *y* koordinátákra 0 és a szélesség/magasság érték között. Hogy, a hörcsögünk rá tudjon lépni, meg kell győződnünk, hogy a répa pozíciója osztható a lépsközzel (```step```). Végül elhelyezzük a répa képét is a grafikus felületen.

In [ ]:
# répa betöltése, méretezése
carrot = pygame.image.load("carrot.png")
carrot = pygame.transform.scale(carrot, (step,step))
carrect = carrot.get_rect()

# a répa pozíciójához szükséges random számok keltése
x_ran = random.randint(0, width)    # random egész szám generálás
y_ran = random.randint(0, height)
x_ran = x_ran - (x_ran % step)      # biztosítjuk, hogy a koordináta osztható a lépésközzel
y_ran = y_ran - (y_ran % step)

# répa közzépontjának elhelyezése a grafikus ablakban
carrect.centerx = x_ran
carrect.centery = y_ran

Az eddigi kóddal képesek vagyunk a hörcsög és a répa képét megjeleníteni a grafikus felületen statikusan. A következő lépés az az, hogy megoldjuk a hörcsög mozgatását. Ehhez definiálunk függvényeket amik a le (```move_down()```), fel (```move_up()```), jobra (```move_right()```) és balra (```move_left()```) mozgásért felelnek. 

Mindegyik függvényben rögzítjük a hörcsög objektumát és a lépésközt mint globális változót. Majd függvénytől függően, utasítjük az objektumot, hogy mozduljon el valamelyik irányba, pl. ```ballrect.move([0,-step])```, ahol az argumentumban a listának első eleme a vízsszintes (*x*) míg a második eleme a függőleges (*y*) elmozdulásért felel. Nyilvánvalóan, ha az egyik koordináta módosítására 0-t írunk, akkor a hörcsög abba az irányba nem fog elmozdulni. 

In [ ]:
def move_up():
    global ballrect, step
    ballrect = ballrect.move([0,-step])

def move_down():
    global ballrect, step
    ballrect = ballrect.move([0,step])

def move_right():
    global ballrect, step
    ballrect = ballrect.move([step,0])

def move_left():
    global ballrect, step
    ballrect = ballrect.move([-step,0])

Eddig definiáltunk négy függvényt, de csak három gombunk van. Mint ahogy írtuk az elején az egyik gombnak így, kettős funkciót kell ellátni. Szabadon lehet választani, hogy melyik két mozgást kötjük össze, de mi a fel/le mozgást rendeljük egy gombhoz, mégpedig attól függően, hogy milyen hosszan lesz lenyomva. Ha a lenyomás 1 másodpercnél rövidebb, akkor felfele, ha hosszabb akkor lefele mozdítja el a hörcsögöt.

Ehhez definiáljuk a ```start```, ```stop``` és ```encoder``` függvényeket:

In [2]:
def start():
    global press
    press = time.time()

def end():
    global release
    release = time.time()
    delay = release - press
    func = encoder(delay)
    func()

def encoder(delay):
    if delay < 1:
        return move_up
    else:
        return move_down

A ```start``` függvény rögzíti a gomb lenyomásának időpillanatát a ```press``` globális változóba. Az ```end``` függvény egyrészt rögzíti a gomb elengedésének pillanatát és a ```release``` változóban tárolja majd kiszámolja, hogy mennyi idő telt el a lenyomás és az elengedés között és a ```delay``` változóba menti. Másrészt, a mért idő alapján az ```encoder``` függvény eldönti, hogy fel ```move_up``` vagy le ```move_down``` irányítsa a keraktert. Ha a lenyomás hossza kevesebb mint 1 másodperc akkor az előzőt teszi, ha több akkor az utóbbit. 

Miután elvégeztük azon függvények definiálását, amiket a gomb lenyomásával szeretnénk aktiválni, a gomb megfelelő funkcióihoz hozzá is kell rendelnünk. Először a fel/le irányt rendeljük a gombhoz a ```start``` és ```end``` függvényeken keresztül.

In [ ]:
up_down.when_pressed = start
up_down.when_released = end

Most értünk el a fő részhez, ahol a játék vezérlése történik. Először is egy végtelen ```while``` ciklust indítunk. A *pygame* gondoskodik arról, hogy egyfolytában figyelje a billentyűzetet, az egeret és egyéb a számítógépre csatolt és definiált eszközöket, és az ezeken bekövetkezett állapotváltozást 'event'-ekbe mentse. 

Az '1. Rész'-ben egy listába gyűjtjük az adott ciklusban bekövetkezett eseményeket, ```pygame.event.get()``` és végig lépkedünk rajta egy ```for``` ciklussal. Amennyiben az egyik esemény típusa megegyezik a ```pygame.QUIT``` definícióval (ami annak felel meg, hogy az ablak piros X gombjára kattintottunk), a python leállítja a játékot és bezárja azt, ```sys.exit()```.

A '2. és 3. Rész'-ben a jobb (```right```) és a bal (```left```) gombokat figyeljük, hogy megnyomták-e az ```is_pressed``` metódussal. Ha igen, meghívjuk a megfelelő mozgatás függvényt. Igazából ezt a két mozgást is definiálhattuk volna a gombok ```when_pressed``` metódusával, de most egy másik megközelítés mellett döntöttünk. 

A '4. Rész' felel azért, hogy a hörcsög a mozgatásokkal ne tudjon lemenni a grafikus felületről. Mindezt úgy tesszük meg, hogy leellenőrizzük, hogy a ```ballrect``` objektum koordinátái (```left```, ```right```, ```top```, ```bottom```) még a grafikus kijelző méretein belül vannak-e (x > 0, x < width és y > 0, y < height). Ha valamelyik korláton készül túllőpni, akkor ezt megakadályozzuk avval, hogy a pozícióját a korláttal tesszük egyenlővé. 

In [ ]:
while 1:
    # 1.Rész: ellenőrizni, milyen események következtek be és végig menni rajtuk
    for event in pygame.event.get():
        print(event)
        # Ha az esemény az ablak bezárása a piros X gombbal, akkor kilépni a játékból
        if event.type == pygame.QUIT:
            sys.exit()
    
    # 2. Rész: Ha a jobb gomb lett lenyomva, mennyünk jobbra
    if right.is_pressed:#If button is pressed, move beachball
        move_right()
    
    # 3. Rész: Ha a bal gomb lett lenyomva, mennyünk balra
    if left.is_pressed:
        move_left()
   
    # 4. Rész: Ellenőrizni, hogy a hörcsög lement-e a grafikus felületről
    if ballrect.left < 0:
        ballrect.left = 0
    if ballrect.right > width: #Move beachball up
        ballrect.right = width
    if ballrect.top < 0:
        ballrect.top = 0
    if ballrect.bottom > height: #Move beachball down
        ballrect.bottom = height    
    
    # 5. Rész: a grafikus felületünk háttere legyen fehér
    screen.fill(white)#Set background of canvas
    screen.blit(carrot, carrect)
    screen.blit(ball, ballrect)
    
    # 6. Rész: Ellenőrízni, hogy a hörcsög megtalálta-e a répát, ha igen, üzenetírás 
    if ballrect.centerx == carrect.centerx and ballrect.centery == carrect.centery:
        text_surface = font.render("Hm, Yummy!", True, (255, 0, 0))
        screen.blit(text_surface, (width/2-30, height/2))
    
    # 7. rész: Az objektumok megjelenítése
    pygame.display.flip()
    time.sleep(0.1)#Wait for 100ms before next button press

Mindeddig igazából nem rajzoltattunk ki semmit a grafikus felületre csak az objektumok módosítására adtunk ki parancsot. Az első dolog, '5. Rész' amit kirajzoltathatunk az a háttér, ```screen.fill()```. Ugyanis ha később rajzoltatjuk le, akkor elfedi az előzőleg lerajzolt objektumoket és csak, a mi esetünkben, fehér hátteret látunk majd. A háttér után jeleníthetjük meg a képeinket. Ezt a ```screen.blit(obj, obj_rect)``` paranccsal tehetjük meg. Minden egyes ciklusban blitteljük, újrarajzoljuk az argumentumban megadott objektumot (obj), a mellé megadott pozícióadatokkal (obj_rect). 

A '6. Rész'-ben leellenőrízzük, hogy a hörcsög megtalálta-e a répát. Ezt mindkét objektum közepének *x* és *y* koordinátájának összehasonlításával tudjuk elérni. Ha azok megegyeznek, ```ballrect.centerx == carrect.centerx and ballrect.centery == carrect.centery``` akkor létrehozunk egy szövegmezőt, ```font.render("Hm, Yummy!", True, (255, 0, 0))```, aminek az első argumentuma maga a megjelenítendő szöveg, a második argumentumban megadhatjük, hogy a szöveg belesimuljon-e a környezetbe és végül a szöveg színét adjuk meg RGB kódban. Ha sikerült létrehozni a szöveg objektumot, azt meg is kell jelentetni, amit szintén blittelünk, ```screen.blit(text_surface, (width/2-30, height/2))```.

Végül, '7. Rész', mindent frissítünk, ```pygame.display.flip()```, a grafikus kijelzőnkön, majd a következő ciklus előtt várunk 100 millimásodpercet, ```time.sleep(0.1)```.

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```button_hamster.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```button_hamster.py```-t elmentettük. Ott begépelve a ```python button_hamster.py``` parancsot, letesztelhetjük a programunk működését. Elindítás után meg kell jelennie a grafikus kijelzőnek egy hörcsög képpel a közepén és egy répával valamerre. A bekötött gombjainkkal tudjuk mozgatni a hörcsögöt a grafikus felületen. Próbáljuk ki az összes beprogramozott gomb funkciót.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Labirintust rajzolni a hörcsögnek, ahol a falakon nem tud átmenni.
* Újabb funkciókat hozzáadni, amit pl. két gomb egyszerre való lenyomásával lehet aktiválni.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) time.time leírása - [https://docs.python.org/3/library/time.html](https://docs.python.org/3/library/time.html)

3) Pygame leírása - [https://www.pygame.org/news](https://www.pygame.org/news)

4) random modul leírás - [https://docs.python.org/3/library/random.html](https://docs.python.org/3/library/random.html)

5) Pygame event - [https://www.raspberry-pi-geek.com/Archive/2014/05/Pygame-modules-for-interactive-programs](https://www.raspberry-pi-geek.com/Archive/2014/05/Pygame-modules-for-interactive-programs)

6) Ikonok - [https://www.freepik.com/](https://www.freepik.com/)